### Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import os
import joblib

#### Mlflow Autologging 

In [2]:
# Enable MLflow autologging 
mlflow.pytorch.autolog()
mlflow.sklearn.autolog(log_models=True)

### Load Data

In [3]:
# Load the cleaned datasets (from Task 1)
fraud_data = pd.read_csv('C:/Users/Administrator/Documents/kifiya/Week_8/clean_data/merged_data.csv')
creditcard_data = pd.read_csv('C:/Users/Administrator/Documents/kifiya/Week_8/clean_data/Preprocessed_Creditcard_Data.csv')

# Drop unnecessary columns for training
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time', 'user_id', 'device_id', 
                                      'ip_address', 'lower_bound_ip_address', 'upper_bound_ip_address'], errors='ignore')

print('The Merged fraud data')
display(fraud_data.head())
print('credit data')
display(creditcard_data.head())

the merged fraud data


,purchase_value,sex,age,class,transaction_count,hour_of_day,day_of_week,purchase_value_scaled,source_Direct,source_SEO,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,47.0,0,30.0,0.0,1,3,6,0.549607,False,True,...,False,False,False,False,False,False,False,False,False,False
1,15.0,0,34.0,0.0,1,20,2,-1.197335,False,True,...,False,False,False,False,False,False,False,False,False,False
2,44.0,1,29.0,0.0,1,23,5,0.385831,False,False,...,False,False,False,False,False,False,False,False,False,False
3,55.0,0,30.0,0.0,1,16,5,0.986342,True,False,...,False,False,False,False,False,False,False,False,False,False
4,51.0,0,37.0,0.0,1,4,1,0.767974,False,True,...,False,False,False,False,False,False,False,False,False,False


credit data


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,time_in_days,Amount_scaled
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.000000,0.244200
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,0.000000,-0.342584
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,0.000012,1.158900
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.000012,0.139886
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,0.000023,-0.073813


## Prepare data for training

In [4]:
# Separate features and targets for Fraud Data
X_fraud = fraud_data.drop(columns=['class'])  # Feature set
y_fraud = fraud_data['class']  # Target

# Separate features and targets for Credit Card Data
X_credit = creditcard_data.drop(columns=['Class'])  # Feature set
y_credit = creditcard_data['Class']  # Target

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_fraud, y_fraud = smote.fit_resample(X_fraud, y_fraud)
X_credit, y_credit = smote.fit_resample(X_credit, y_credit)

#for scikit-learn models
y_credit_sci=y_credit.values.ravel()
y_fraud_sci=y_fraud.values.ravel()
X_fraud_sci=X_fraud
X_credit_sci=X_credit



# Train-Test Split for both datasets
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.25, random_state=42)
X_credit_train, X_credit_test, y_credit_train, y_credit_test = train_test_split(X_credit, y_credit, test_size=0.25, random_state=42)

# train split for scikit_learn models
X_fraud_train_sci, X_fraud_test_sci, y_fraud_train_sci, y_fraud_test_sci = train_test_split(X_fraud_sci, y_fraud_sci, test_size=0.25, random_state=42)
X_credit_train_sci, X_credit_test_sci, y_credit_train_sci, y_credit_test_sci = train_test_split(X_credit_sci, y_credit_sci, test_size=0.25, random_state=42)

# Normalize the data (Standard Scaling)
scaler = StandardScaler()
X_fraud_train = scaler.fit_transform(X_fraud_train)
X_fraud_test = scaler.transform(X_fraud_test)
X_credit_train = scaler.fit_transform(X_credit_train)
X_credit_test = scaler.transform(X_credit_test)

# Convert the datasets into PyTorch tensors
X_fraud_train_tensor = torch.tensor(X_fraud_train, dtype=torch.float32)
y_fraud_train_tensor = torch.tensor(y_fraud_train.values, dtype=torch.float32)
X_fraud_test_tensor = torch.tensor(X_fraud_test, dtype=torch.float32)
y_fraud_test_tensor = torch.tensor(y_fraud_test.values, dtype=torch.float32)

X_credit_train_tensor = torch.tensor(X_credit_train, dtype=torch.float32)
y_credit_train_tensor = torch.tensor(y_credit_train.values, dtype=torch.float32)
X_credit_test_tensor = torch.tensor(X_credit_test, dtype=torch.float32)
y_credit_test_tensor = torch.tensor(y_credit_test.values, dtype=torch.float32)

# DataLoader for batching
batch_size = 16
train_loader_fraud = torch.utils.data.DataLoader(TensorDataset(X_fraud_train_tensor, y_fraud_train_tensor), batch_size=batch_size, shuffle=True)
test_loader_fraud = torch.utils.data.DataLoader(TensorDataset(X_fraud_test_tensor, y_fraud_test_tensor), batch_size=batch_size)

train_loader_credit = torch.utils.data.DataLoader(TensorDataset(X_credit_train_tensor, y_credit_train_tensor), batch_size=batch_size, shuffle=True)
test_loader_credit = torch.utils.data.DataLoader(TensorDataset(X_credit_test_tensor, y_credit_test_tensor), batch_size=batch_size)


2024/10/26 14:12:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '38faa4d05523462aa5f0f8db227604d6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/10/26 14:12:46 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/10/26 14:12:46 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/10/26 14:12:46 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/10/26 14:12:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signatu

#### Early stop function

In [5]:
# Early stopping mechanism
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, loss):
        if self.best_loss is None:
            self.best_loss = loss
        elif loss > self.best_loss + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = loss
            self.counter = 0


### Defining deep learning models

In [6]:
# Model definitions (MLP, CNN, RNN, LSTM)
class MLPModel(nn.Module):
    def __init__(self, input_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

class CNNModel(nn.Module):
    def __init__(self, input_size):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * input_size, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.sigmoid(self.fc1(x))
        return x

class RNNModel(nn.Module):
    def __init__(self, input_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        h0 = torch.zeros(1, x.size(0), 32)  # Initial hidden state
        out, _ = self.rnn(x, h0)
        out = torch.sigmoid(self.fc(out[:, -1, :]))
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        h0 = torch.zeros(1, x.size(0), 32)  # Initial hidden state
        c0 = torch.zeros(1, x.size(0), 32)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))
        out = torch.sigmoid(self.fc(out[:, -1, :]))
        return out

## Train ML models function

In [7]:

# Scikit-learn classifiers
def train_sklearn_model(model, X_train, y_train):
    model.fit(X_train, y_train)

## Trian DL models function with MLflow tracking

In [8]:
# Training loop with MLflow tracking
def train_model(model, train_loader, optimizer, criterion, num_epochs=15, patience=3, model_name="model"):
    early_stopper = EarlyStopping(patience=patience)
    model.train()
    
    with mlflow.start_run(run_name=model_name):
        for epoch in range(num_epochs):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()  # Clear gradients
                y_pred = model(X_batch).squeeze()  # Forward pass
                loss = criterion(y_pred, y_batch)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update weights
                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
            
            # Log loss for each epoch
            mlflow.log_metric('loss', avg_loss, step=epoch)

            # Early stopping
            early_stopper(avg_loss)
            if early_stopper.early_stop:
                print("Early stopping triggered!")
                break

### Evaluate DL models

In [9]:
# Model evaluation
# Updated PyTorch model evaluation function
def evaluate_model(model, test_loader):
    model.eval()  # Set to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch).squeeze()
            preds = (y_pred > 0.5).float()  # Convert probabilities to 0/1
            all_preds.extend(preds.numpy())
            all_labels.extend(y_batch.numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    print(f'Accuracy: {accuracy:.4f}')
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

### Evaluate ML models with MLflow Tracking

In [10]:
def evaluate_sklearn_model(model, X_test, y_test, model_name):
    with mlflow.start_run(run_name=model_name):
        # Get predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # Log metrics to MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Print metrics
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

## Initializing models

In [11]:
# Initialize models for both fraud and credit datasets
input_size_fraud = X_fraud_train.shape[1]
input_size_credit = X_credit_train.shape[1]

mlp_model_fraud = MLPModel(input_size_fraud)
cnn_model_fraud = CNNModel(input_size_fraud)
rnn_model_fraud = RNNModel(input_size_fraud)
lstm_model_fraud = LSTMModel(input_size_fraud)

mlp_model_credit = MLPModel(input_size_credit)
cnn_model_credit = CNNModel(input_size_credit)
rnn_model_credit = RNNModel(input_size_credit)
lstm_model_credit = LSTMModel(input_size_credit)

# LogisticRegression, RandomForest, GradientBoosting, DecisionTree
logistic_fraud = LogisticRegression(max_iter=1000)
rf_fraud = RandomForestClassifier(n_estimators=100)
gb_fraud = GradientBoostingClassifier(n_estimators=100)
dt_fraud = DecisionTreeClassifier()

logistic_credit = LogisticRegression(max_iter=1000)
rf_credit = RandomForestClassifier(n_estimators=100)
gb_credit = GradientBoostingClassifier(n_estimators=100)
dt_credit = DecisionTreeClassifier()

# Set loss function and optimizer
criterion = nn.BCELoss()

# Train and evaluate each model for both fraud and credit data
models = {
    "MLP_Fraud": mlp_model_fraud,
    "CNN_Fraud": cnn_model_fraud,
    "RNN_Fraud": rnn_model_fraud,
    "LSTM_Fraud": lstm_model_fraud,
    "MLP_Credit": mlp_model_credit,
    "CNN_Credit": cnn_model_credit,
    "RNN_Credit": rnn_model_credit,
    "LSTM_Credit": lstm_model_credit,
    "LogisticRegression_Fraud": logistic_fraud,
    "RandomForest_Fraud": rf_fraud,
    "GradientBoosting_Fraud": gb_fraud,
    "DecisionTree_Fraud": dt_fraud,
    "LogisticRegression_Credit": logistic_credit,
    "RandomForest_Credit": rf_credit,
    "GradientBoosting_Credit": gb_credit,
    "DecisionTree_Credit": dt_credit
}

### Function to save trained models

In [12]:
# saving the models
save_folder = 'C:/Users/Administrator/Documents/kifiya/Week_8/saved_models'
os.makedirs(save_folder, exist_ok=True) 

# Function to save PyTorch models
def save_pytorch_model(model, model_name):
    save_path = os.path.join(save_folder, f'{model_name}.pt')
    torch.save(model.state_dict(), save_path)
    print(f'{model_name} saved at {save_path}')

# Function to save scikit-learn models
def save_sklearn_model(model, model_name):
    save_path = os.path.join(save_folder, f'{model_name}.joblib')
    joblib.dump(model, save_path)
    print(f'{model_name} saved at {save_path}')


## Train, Evaluate and Save models

In [13]:
# Training and evaluating each model with MLflow tracking
# Modify the existing model loop to include saving logic
for model_name, model in models.items():
    if isinstance(model, nn.Module):  # For PyTorch models
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        print(f"\nTraining {model_name}...")
        train_model(model, train_loader_fraud if "Fraud" in model_name else train_loader_credit, optimizer, criterion)
        print(f"Evaluating {model_name}...")
        evaluate_model(model, test_loader_fraud if "Fraud" in model_name else test_loader_credit)
        save_pytorch_model(model, model_name)  # Save PyTorch model
        print('----------------------------------------------------------------------------------------------------------------------------------------------------')
    else:  # For scikit-learn models
        print(f"\nTraining {model_name}...")
        X_train = X_fraud_train_sci if "Fraud" in model_name else X_credit_train_sci
        y_train = y_fraud_train_sci if "Fraud" in model_name else y_credit_train_sci
        X_test = X_fraud_test_sci if "Fraud" in model_name else X_credit_test_sci
        y_test = y_fraud_test_sci if "Fraud" in model_name else y_credit_test_sci
        train_sklearn_model(model, X_train, y_train)
        print(f"Evaluating {model_name}...")
        evaluate_sklearn_model(model, X_test, y_test,model_name)
        save_sklearn_model(model, model_name)  # Save scikit-learn model
        print('-----------------------------------------------------------------------------------------------------------------------------------------------------')


Training MLP_Fraud...
Epoch [1/15], Loss: 0.5104
Epoch [2/15], Loss: 0.4200
Epoch [3/15], Loss: 0.3854
Epoch [4/15], Loss: 0.3716
Epoch [5/15], Loss: 0.3480
Epoch [6/15], Loss: 0.3346
Epoch [7/15], Loss: 0.3249
Epoch [8/15], Loss: 0.3182
Epoch [9/15], Loss: 0.3123
Epoch [10/15], Loss: 0.3079
Epoch [11/15], Loss: 0.3040
Epoch [12/15], Loss: 0.2990
Epoch [13/15], Loss: 0.2954
Epoch [14/15], Loss: 0.2918
Epoch [15/15], Loss: 0.2892
Evaluating MLP_Fraud...
Accuracy: 0.8816
Precision: 0.8816
Recall: 0.8816
F1 Score: 0.8816
MLP_Fraud saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\MLP_Fraud.pt
----------------------------------------------------------------------------------------------------------------------------------------------------

Training CNN_Fraud...
Epoch [1/15], Loss: 0.5665
Epoch [2/15], Loss: 0.5344
Epoch [3/15], Loss: 0.5200
Epoch [4/15], Loss: 0.5080
Epoch [5/15], Loss: 0.4982
Epoch [6/15], Loss: 0.4906
Epoch [7/15], Loss: 0.4839
Epoch [8/15], Loss: 0.

2024/10/26 16:06:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b22ca72d20354ad29f8e9147bd79da42', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Accuracy: 0.9997
Precision: 0.9997
Recall: 0.9997
F1 Score: 0.9997
LSTM_Credit saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\LSTM_Credit.pt
----------------------------------------------------------------------------------------------------------------------------------------------------

Training LogisticRegression_Fraud...


2024/10/26 16:06:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to co

Evaluating LogisticRegression_Fraud...


...

Accuracy: 0.9776
Precision: 0.9778
Recall: 0.9776
F1 Score: 0.9776
LogisticRegression_Credit saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\LogisticRegression_Credit.joblib
-----------------------------------------------------------------------------------------------------------------------------------------------------

Training RandomForest_Credit...
Evaluating RandomForest_Credit...


2024/10/26 16:21:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b30a037ae47c46d0962eb413a21056c7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Accuracy: 0.9999
Precision: 0.9999
Recall: 0.9999
F1 Score: 0.9999
RandomForest_Credit saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\RandomForest_Credit.joblib
-----------------------------------------------------------------------------------------------------------------------------------------------------

Training GradientBoosting_Credit...
Evaluating GradientBoosting_Credit...


2024/10/26 16:34:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'acc9dfae32104e62aa7230bf8cb9f3b0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Accuracy: 0.9863
Precision: 0.9864
Recall: 0.9863
F1 Score: 0.9863
GradientBoosting_Credit saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\GradientBoosting_Credit.joblib
-----------------------------------------------------------------------------------------------------------------------------------------------------

Training DecisionTree_Credit...
Evaluating DecisionTree_Credit...
Accuracy: 0.9986
Precision: 0.9986
Recall: 0.9986
F1 Score: 0.9986
DecisionTree_Credit saved at C:/Users/Administrator/Documents/kifiya/Week_8/saved_models\DecisionTree_Credit.joblib
-----------------------------------------------------------------------------------------------------------------------------------------------------
